In [1]:
import pandas as pd

In [2]:
# determine voting population by state
v_pop_2016_df = pd.read_csv('data/general-election-2016.csv')
v_pop_2016_df.head()

,State,VEP Total Ballots Counted,VEP Highest Office,VAP Highest Office,Total Ballots Counted,Highest Office,Voting-Eligible Population (VEP),Voting-Age Population (VAP),% Non-citizen,Prison,Probation,Parole,Total Ineligible Felon,Overseas Eligible,State Abv
0,United States,NaN,NaN,NaN,NaN,NaN,2.315566e+08,251107404.0,0.08,1493706.0,2298993.0,461472.0,3242614.08,4739595.99,US
1,Alabama,NaN,NaN,NaN,NaN,NaN,3.606103e+06,3773719.0,0.03,31691.0,53640.0,8097.0,69826.40,NaN,AL
2,Alaska,NaN,NaN,NaN,NaN,NaN,5.195009e+05,554802.0,0.04,5216.0,7077.0,2210.0,11389.12,NaN,AK
3,Arizona,NaN,NaN,NaN,NaN,NaN,4.738332e+06,5332446.0,0.10,37516.0,73232.0,7502.0,86027.92,NaN,AZ
4,Arkansas,NaN,NaN,NaN,NaN,NaN,2.148441e+06,2290976.0,0.04,17756.0,28192.0,21743.0,55286.52,NaN,AR


In [3]:
# Have to estimate the voting population per state using 2012 data (this is probably terrible--sorry, Chris A!)
v_pop_2012_df = pd.read_csv('data/general-election-2012.csv')
v_pop_2016_df['vep_ho'] = v_pop_2016_df['Voting-Eligible Population (VEP)'] * v_pop_2012_df['VEP Highest Office']
v_pop_2016_df[['State', 'vep_ho']].head()

,State,vep_ho
0,United States,1.343028e+08
1,Alabama,2.113176e+06
2,Alaska,3.049470e+05
3,Arizona,2.492363e+06
4,Arkansas,1.089259e+06


In [4]:
# Merge on electoral votes
state_df = pd.read_csv('data/electoral-votes.csv', index_col='State')
state_df.head()

,Electoral Votes
State,
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [5]:
pop_df = state_df.merge(v_pop_2016_df[['State', 'State Abv', 'vep_ho']], how='outer', left_index=True, right_on='State')
pop_df.set_index('State', inplace=True)
pop_df.head()

,Electoral Votes,State Abv,vep_ho
State,,,
Alabama,9.0,AL,2.113176e+06
Alaska,3.0,AK,3.049470e+05
Arizona,11.0,AZ,2.492363e+06
Arkansas,6.0,AR,1.089259e+06
California,55.0,CA,1.392862e+07


In [6]:
# I can sum the electoral votes for the country and I should get something familiar.
pop_df.loc[pop_df['State Abv'] == 'US', 'Electoral Votes'] = pop_df.loc[pop_df['State Abv'] != 'US', 'Electoral Votes'].sum()
pop_df.loc[pop_df['State Abv'] == 'US']

,Electoral Votes,State Abv,vep_ho
State,,,
United States,538.0,US,1.343028e+08


In [7]:
# Now look at people's responses
picks_df = pd.read_csv('data/picks.csv', na_values=['TBD'], index_col='State')
picks_df.head()

,Aaron,Austin,Bill,Bret,Julia,Dale,David,Dawn,Diane,Erica,...,Luke,Matt,Max,Neil,Phil C,Phil K,Ron,Sara,Sasha,OUTCOME
State,,,,,,,,,,,,,,,,,,,,,
Alabama,Trump 20+,NaN,Trump 20+,Trump 10+,Trump 10+,Trump 20+,NaN,Trump 10+,Trump 20+,Trump 20+,...,NaN,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,NaN
Alaska,Trump 10+,NaN,Trump 10+,Trump 0+,Trump 5+,Trump 10+,NaN,Trump 10+,Trump 10+,Trump 5+,...,NaN,Trump 5+,Trump 5+,Trump 5+,Trump 5+,Trump 5+,Trump 10+,Trump 5+,Trump 5+,NaN
Arizona,Trump 0+,NaN,Trump 0+,Trump 5+,Clinton 0+,Trump 0+,NaN,Trump 0+,Trump 10+,Trump 0+,...,NaN,Trump 0+,Trump 0+,Trump 0+,Trump 0+,Trump 0+,Trump 5+,Trump 0+,Trump 0+,NaN
Arkansas,Trump 10+,NaN,Trump 10+,Trump 5+,Trump 10+,Trump 20+,NaN,Trump 20+,Trump 20+,Trump 10+,...,NaN,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 10+,Trump 10+,NaN
California,Clinton 20+,NaN,Clinton 10+,Clinton 20+,Clinton 20+,Clinton 20+,NaN,Clinton 0+,Clinton 20+,Clinton 20+,...,NaN,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 10+,Clinton 20+,NaN


In [8]:
# Drop people who have NaN or TBD selections anywhere because I don't know what to do with them.
picks_df.dropna(axis='columns', how='any', inplace=True)
picks_df.head()

,Aaron,Bill,Bret,Julia,Dale,Diane,Erica,Glenn,Jessica,John,Kim,Liz,Matt,Max,Neil,Phil C,Phil K,Ron,Sara,Sasha
State,,,,,,,,,,,,,,,,,,,,
Alabama,Trump 20+,Trump 20+,Trump 10+,Trump 10+,Trump 20+,Trump 20+,Trump 20+,Clinton 0+,Trump 10+,Trump 20+,Trump 10+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+
Alaska,Trump 10+,Trump 10+,Trump 0+,Trump 5+,Trump 10+,Trump 10+,Trump 5+,Trump 5+,Trump 5+,Trump 10+,Trump 10+,Trump 0+,Trump 5+,Trump 5+,Trump 5+,Trump 5+,Trump 5+,Trump 10+,Trump 5+,Trump 5+
Arizona,Trump 0+,Trump 0+,Trump 5+,Clinton 0+,Trump 0+,Trump 10+,Trump 0+,Trump 5+,Trump 0+,Trump 0+,Trump 0+,Trump 0+,Trump 0+,Trump 0+,Trump 0+,Trump 0+,Trump 0+,Trump 5+,Trump 0+,Trump 0+
Arkansas,Trump 10+,Trump 10+,Trump 5+,Trump 10+,Trump 20+,Trump 20+,Trump 10+,Clinton 5+,Trump 5+,Trump 10+,Trump 10+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 20+,Trump 10+,Trump 10+
California,Clinton 20+,Clinton 10+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 20+,Clinton 10+,Clinton 20+


In [9]:
# Convert to some sort of margin
margin_map = {
    'Trump 20+': -.225,
    'Trump 10+': -.15,
    'Trump 5+': -.075,
    'Trump 0+': -.025,
    'Clinton 0+': .025,
    'Clinton 5+': .075,
    'Clinton 10+': .15,
    'Clinton 20+': .225,
}
picks_pct_df = picks_df.applymap(lambda x: margin_map[x])
picks_pct_df.head()

,Aaron,Bill,Bret,Julia,Dale,Diane,Erica,Glenn,Jessica,John,Kim,Liz,Matt,Max,Neil,Phil C,Phil K,Ron,Sara,Sasha
State,,,,,,,,,,,,,,,,,,,,
Alabama,-0.225,-0.225,-0.150,-0.150,-0.225,-0.225,-0.225,0.025,-0.150,-0.225,-0.150,-0.225,-0.225,-0.225,-0.225,-0.225,-0.225,-0.225,-0.225,-0.225
Alaska,-0.150,-0.150,-0.025,-0.075,-0.150,-0.150,-0.075,-0.075,-0.075,-0.150,-0.150,-0.025,-0.075,-0.075,-0.075,-0.075,-0.075,-0.150,-0.075,-0.075
Arizona,-0.025,-0.025,-0.075,0.025,-0.025,-0.150,-0.025,-0.075,-0.025,-0.025,-0.025,-0.025,-0.025,-0.025,-0.025,-0.025,-0.025,-0.075,-0.025,-0.025
Arkansas,-0.150,-0.150,-0.075,-0.150,-0.225,-0.225,-0.150,0.075,-0.075,-0.150,-0.150,-0.225,-0.225,-0.225,-0.225,-0.225,-0.225,-0.225,-0.150,-0.150
California,0.225,0.150,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.225,0.150,0.225


In [12]:
# How many electoral votes per person?
picks_dem_df = picks_pct_df > 0
ev_df = picks_dem_df.mul(pop_df.loc[pop_df.index != 'United States', 'Electoral Votes'], axis='index')
ev_df -= (~picks_dem_df).mul(pop_df.loc[pop_df.index != 'United States', 'Electoral Votes'], axis='index')

In [15]:
# How many popular votes per person? (this is probably wrong--sorry, Chris A!)
pv_df = picks_pct_df.mul(pop_df.loc[pop_df.index != 'United States', 'vep_ho'], axis='index')

In [18]:
# People with Clinton winning electoral college
ev_df.sum(axis='index').sort_values()

Dale      -104.0
Phil K       6.0
Liz         18.0
Phil C      50.0
Bret        56.0
Aaron       76.0
Sasha       78.0
Bill        90.0
Matt       108.0
Max        108.0
Neil       108.0
Ron        108.0
Erica      108.0
John       120.0
Diane      124.0
Sara       126.0
Kim        126.0
Julia      142.0
Jessica    156.0
Glenn      226.0
dtype: float64

In [19]:
# People with Clinton winning popular vote
pv_df.sum(axis='index').sort_values()

Dale      -2.989665e+05
Bill       2.427051e+06
Phil C     3.127778e+06
Diane      3.175207e+06
Phil K     3.267688e+06
Sara       3.428570e+06
Ron        3.436375e+06
Sasha      4.099412e+06
Erica      4.100519e+06
Matt       4.146827e+06
Liz        4.196695e+06
Neil       4.758005e+06
Jessica    5.673229e+06
Max        6.011226e+06
John       6.208564e+06
Bret       6.412500e+06
Aaron      6.639357e+06
Julia      7.028478e+06
Kim        7.407334e+06
Glenn      9.325570e+06
dtype: float64